# Tweets streaming - Consumer
Read Stream into dataframe, add timestamp, sentiment and price, save aggregated window

Version 12: we need to first save in memory and only then in parquet as a workaround

In [0]:
#!pip install textblob
#!pip install pycountry
#!pip install bs4

Collecting textblob
 Downloading textblob-0.15.3-py2.py3-none-any.whl (636 kB)
▌ | 10 kB 18.4 MB/s eta 0:00:01 |█ | 20 kB 22.5 MB/s eta 0:00:01 |█▌ | 30 kB 13.1 MB/s eta 0:00:01 |██ | 40 kB 9.9 MB/s eta 0:00:01 |██▋ | 51 kB 10.1 MB/s eta 0:00:01 |███ | 61 kB 8.1 MB/s eta 0:00:01 |███▋ | 71 kB 8.4 MB/s eta 0:00:01 |████▏ | 81 kB 8.6 MB/s eta 0:00:01 |████▋ | 92 kB 8.1 MB/s eta 0:00:01 |█████▏ | 102 kB 7.5 MB/s eta 0:00:01 |█████▋ | 112 kB 7.5 MB/s eta 0:00:01 |██████▏ | 122 kB 7.5 MB/s eta 0:00:01 |██████▊ | 133 kB 7.5 MB/s eta 0:00:01 |███████▏ | 143 kB 7.5 MB/s eta 0:00:01 |███████▊ | 153 kB 7.5 MB/s eta 0:00:01 |████████▎ | 163 kB 7.5 MB/s eta 0:00:01 |████████▊ | 174 kB 7.5 MB/s eta 0:00:01 |█████████▎ | 184 kB 7.5 MB/s eta 0:00:01 |█████████▉ | 194 kB 7.5 MB/s eta 0:00:01 |██████████▎ | 204 kB 7.5 MB/s eta 0:00:01 |██████████▉ | 215 kB 7.5 MB/s eta 0:00:01 |███████████▎ | 225 kB 7.5 MB/s eta 0:00:01 |███████████▉ | 235 kB 7.5 MB/s eta 0:00:01 |████████████▍ | 245 kB 7.5 MB/s eta 0:00:01 |████████████▉ | 256 kB 7.5 MB/s eta 0:00:01 |█████████████▍ | 266 kB 7.5 MB/s eta 0:00:01 |██████████████ | 276 kB 7.5 MB/s eta 0:00:01 |██████████████▍ | 286 kB 7.5 MB/s eta 0:00:01 |███████████████ | 296 kB 7.5 MB/s eta 0:00:01 |███████████████▍ | 307 kB 7.5 MB/s eta 0:00:01 |████████████████ | 317 kB 7.5 MB/s eta 0:00:01 |████████████████▌ | 327 kB 7.5 MB/s eta 0:00:01 |█████████████████ | 337 kB 7.5 MB/s eta 0:00:01 |█████████████████▌ | 348 kB 7.5 MB/s eta 0:00:01 |██████████████████ | 358 kB 7.5 MB/s eta 0:00:01 |██████████████████▌ | 368 kB 7.5 MB/s eta 0:00:01 |███████████████████ | 378 kB 7.5 MB/s eta 0:00:01 |███████████████████▋ | 389 kB 7.5 MB/s eta 0:00:01 |████████████████████ | 399 kB 7.5 MB/s eta 0:00:01 |████████████████████▋ | 409 kB 7.5 MB/s eta 0:00:01 |█████████████████████ | 419 kB 7.5 MB/s eta 0:00:01 |█████████████████████▋ | 430 kB 7.5 MB/s eta 0:00:01 |██████████████████████▏ | 440 kB 7.5 MB/s eta 0:00:01 |██████████████████████▋ | 450 kB 7.5 MB/s eta 0:00:01 |███████████████████████▏ | 460 kB 7.5 MB/s eta 0:00:01 |███████████████████████▊ | 471 kB 7.5 MB/s eta 0:00:01 |████████████████████████▏ | 481 kB 7.5 MB/s eta 0:00:01 |████████████████████████▊ | 491 kB 7.5 MB/s eta 0:00:01 |█████████████████████████▎ | 501 kB 7.5 MB/s eta 0:00:01 |█████████████████████████▊ | 512 kB 7.5 MB/s eta 0:00:01 |██████████████████████████▎ | 522 kB 7.5 MB/s eta 0:00:01 |██████████████████████████▊ | 532 kB 7.5 MB/s eta 0:00:01 |███████████████████████████▎ | 542 kB 7.5 MB/s eta 0:00:01 |███████████████████████████▉ | 552 kB 7.5 MB/s eta 0:00:01 |████████████████████████████▎ | 563 kB 7.5 MB/s eta 0:00:01 |████████████████████████████▉ | 573 kB 7.5 MB/s eta 0:00:01 |█████████████████████████████▍ | 583 kB 7.5 MB/s eta 0:00:01 |█████████████████████████████▉ | 593 kB 7.5 MB/s eta 0:00:01 |██████████████████████████████▍ | 604 kB 7.5 MB/s eta 0:00:01 |██████████████████████████████▉ | 614 kB 7.5 MB/s eta 0:00:01 |███████████████████████████████▍| 624 kB 7.5 MB/s eta 0:00:01 |████████████████████████████████| 634 kB 7.5 MB/s eta 0:00:01 |████████████████████████████████| 636 kB 7.5 MB/s 
Collecting nltk>=3.1
 Downloading nltk-3.6.2-py3-none-any.whl (1.5 MB)
▎ | 10 kB 29.0 MB/s eta 0:00:01 |▌ | 20 kB 34.9 MB/s eta 0:00:01 |▊ | 30 kB 40.0 MB/s eta 0:00:01 |█ | 40 kB 43.1 MB/s eta 0:00:01 |█▏ | 51 kB 32.4 MB/s eta 0:00:01 |█▍ | 61 kB 35.2 MB/s eta 0:00:01 |█▋ | 71 kB 27.7 MB/s eta 0:00:01 |█▉ | 81 kB 29.8 MB/s eta 0:00:01 |██ | 92 kB 17.5 MB/s eta 0:00:01 |██▎ | 102 kB 16.0 MB/s eta 0:00:01 |██▌ | 112 kB 16.0 MB/s eta 0:00:01 |██▊ | 122 kB 16.0 MB/s eta 0:00:01 |███ | 133 kB 16.0 MB/s eta 0:00:01 |███▏ | 143 kB 16.0 MB/s eta 0:00:01 |███▍ | 153 kB 16.0 MB/s eta 0:00:01 |███▋ | 163 kB 16.0 MB/s eta 0:00:01 |███▉ | 174 kB 16.0 MB/s eta 0:00:01 |████ | 184 kB 16.0 MB/s eta 0:00:01 |████▎ | 194 kB 16.0 MB/s eta 0:00:01 |████▌ | 204 kB 16.0 MB/s eta 0:00:01

In [0]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F
from textblob import TextBlob
import re
import pycountry
from datetime import datetime
import requests
import pandas as pd
import json
import time
from bs4 import BeautifulSoup

In [0]:
# Funktions to split away the timestamp from the tweet message
@udf
def split_start(text):
  stamp = text[:-27]
  return stamp
@udf
def split_back(text):
  stamp = text[-21:-4]     # -4 because reading it without milliseconds 
  return stamp


In [0]:
# Additional cleaning and splitting of the incoming text
def preprocessing(lines):
    words = lines.select(explode(split(lines.value, "t_end")).alias("word"))
    words = words.na.replace('', None)
    words = words.na.drop()
    words = words.withColumn('word', F.regexp_replace('word', r'http\S+', ''))
    words = words.withColumn('word', F.regexp_replace('word', '@\w+', ''))
    words = words.withColumn('word', F.regexp_replace('word', '#', ''))
    words = words.withColumn('word', F.regexp_replace('word', 'RT', ''))
    words = words.withColumn('word', F.regexp_replace('word', ':', ''))
    words = words.withColumn('tweet_txt', split_start('word'))
    words = words.withColumn('stamp', split_back('word'))
    return words
  

In [0]:
# Apply Textblob Sentiment Analysis
def polarity_detection(text):
    return TextBlob(text).sentiment.polarity

def text_classification(words):
    # polarity detection
    polarity_detection_udf = udf(polarity_detection, StringType())
    words = words.withColumn("polarity", polarity_detection_udf("tweet_txt"))
    return words


In [0]:
# Language Detection
def language_detection(text):
    try:
      iso_code = TextBlob(text).detect_language()
      language = pycountry.languages.get(alpha_2=iso_code)
      language_name = language.name
    except:
      language_name = 'no language detected'
    return language_name

def apply_language(words):
    # language detection
    language_detection_udf = udf(language_detection, StringType())
    words = words.withColumn("language", language_detection_udf("tweet_txt"))
    return words
  

In [0]:
# Funtcion to call the current bitcoint price
@udf
def get_actual_crypto_price(crypto):
  base_url = 'https://coinmarketcap.com'
  request = requests.get(base_url)
  soup = BeautifulSoup(request.content, 'html.parser')
  data = soup.find('script', id="__NEXT_DATA__", type="application/json")
  coins = {}

  coin_data = json.loads(data.contents[0])
  listings = coin_data['props']['initialState']['cryptocurrency']['listingLatest']['data']

  for i in listings:
    crypto_curr = i['name']
    if crypto_curr.lower() == crypto.lower():
      evaluate_price = i['quotes'][2]
      #print(evaluate_price)
      coins[str(i['id'])] = i['slug']
      coins['currency'] = evaluate_price['name']
      coins['actual_price'] = evaluate_price['price']
      coins['percentChange24h'] = evaluate_price['percentChange24h']
  return coins['actual_price']


In [0]:
# Create Spark session
spark = SparkSession.builder.appName("TwitterSentimentAnalysis").getOrCreate()

# Read the tweet data from socket
lines = spark.readStream.format("socket") \
        .option("host", "localhost") \
        .option("port", 9997) \
        .load()

# Preprocess the data
words = preprocessing(lines)

# Apply column with text classification to define polarity
words = text_classification(words)

# Add column with language
words = apply_language(words)
                         
# Add a column with the currency lable 'bitcoin'
words = words.withColumn("crypto", lit('bitcoin'))


In [0]:
# Filter all tweets in English
words = words.filter(words.language == "English")

# Filter out all tweets with polarity '0'
words = words.filter(words.polarity != "0.0")


In [0]:
# Re-format timestamp from string to TimestampType
format = '%Y-%m-%dT%H%M%S'
time_udf = udf(lambda x: datetime.strptime(x, format), TimestampType())
words = words.withColumn('t_stamp', time_udf('stamp'))

# Re-format polarity to float
words = words.withColumn('polarity', col('polarity').cast('float'))


In [0]:
# Create a 6 second window as basis for all subsequent analysis (since version 11 with watermark)
windowedStream = words.withWatermark('t_stamp', '6 seconds') \
                      .groupBy(window('t_stamp', '6 seconds', '6 seconds'))

# previous versions without watermark:
#windowedStream = words.groupBy(window('t_stamp', '6 seconds', '6 seconds'))

# Aggregate the figures we need and add the bitcoin price
aggregationsStream = windowedStream \
        .agg(count('tweet_txt').alias('count_tweets') \
           , avg('polarity').alias('pol_avg') \
           , get_actual_crypto_price(first(col('crypto'))).cast('float').alias('window_price') \
           , max('t_stamp').alias('t_stamp')
           )


In [0]:
# Both the words-Stream as well as the aggregationsStream can be displayed
# display(words)
# display(aggregationsStream)

In [0]:
# Try to write the aggregationStream to parquet with 'append mode' (with watermark included in command 12)
# --> this did not work: parquet file is created with structure but file has no records
# we did not find a solution to make this work
'''
aggregationsStream \
    .writeStream \
    .format("parquet")\
    .option("path", "/experiment_results /original")\
    .option("checkpointLocation", "/ experiment_results /check")\
    .outputMode("append")\
    .start()
'''

In [0]:
# Write to memory with 'append mode' to overcome the parquet problem
streamingETLQuery = aggregationsStream \
  .writeStream \
  .format("memory") \
  .queryName("aggDF") \
  .outputMode("append")\
  .start()


In [0]:
# Count the records in memory
spark.sql("select count(*) from aggDF").show()

+--------+
count(1)|
+--------+
 17|
+--------+

In [0]:
# Display the records in memory
spark.sql("select * from aggDF")\
  .sort('t_stamp')\
  .show() 


+--------------------+------------+-------------------+------------+-------------------+
 window|count_tweets| pol_avg|window_price| t_stamp|
+--------------------+------------+-------------------+------------+-------------------+
{2021-06-18 19:55...| 8|0.32475785398855805| 35425.062|2021-06-18 19:55:53|
{2021-06-18 19:55...| 11| 0.294422254643657| 35425.062|2021-06-18 19:55:58|
{2021-06-18 19:56...| 11|0.09052319737913256| 35425.062|2021-06-18 19:56:05|
{2021-06-18 19:56...| 17|0.25537307964473527| 35425.062|2021-06-18 19:56:11|
{2021-06-18 19:56...| 15|0.11737926964027186| 35425.062|2021-06-18 19:56:17|
{2021-06-18 19:56...| 15|0.14881313499063253| 35425.062|2021-06-18 19:56:23|
{2021-06-18 19:56...| 11|0.15214646743102508| 35425.062|2021-06-18 19:56:29|
{2021-06-18 19:56...| 15| 0.2853124099317938| 35425.062|2021-06-18 19:56:35|
{2021-06-18 19:56...| 14|0.09170317169212337| 35425.062|2021-06-18 19:56:41|
{2021-06-18 19:56...| 15|0.15865079474945862| 35425.062|2021-06-18 19:56:47|
{2021-06-18 19:56...| 11| 0.2837055657185953| 35425.062|2021-06-18 19:56:53|
{2021-06-18 19:56...| 16| 0.2677955897524953| 35425.062|2021-06-18 19:56:59|
{2021-06-18 19:57...| 9|0.20267857155866092| 35425.062|2021-06-18 19:57:05|
{2021-06-18 19:57...| 10|0.23000000156462191| 35425.062|2021-06-18 19:57:11|
{2021-06-18 19:57...| 12| 0.2751129857691315| 35425.062|2021-06-18 19:57:17|
{2021-06-18 19:57...| 7|0.19107143155166081| 35425.062|2021-06-18 19:57:23|
{2021-06-18 19:57...| 6| 0.2654431238770485| 35425.062|2021-06-18 19:57:29|
+--------------------+------------+-------------------+------------+-------------------+

In [0]:
# Persistently store the result in a parquet file
spark.sql("select * from aggDF").write.parquet('dbfs:/FileStore/original/tweet_sparksql')

In [0]:
#%fs ls dbfs:/FileStore/

In [0]:
#%fs rm -r dbfs:/FileStore/original/

In [0]:
#%fs rm -r dbfs:/local_disk0/tmp/